# Big Data Modeling and Management 2021


## 🚚 BDMM Third Homework Assignment 🚚 

### CRUD operations

In [141]:
#!pip install mysql-connector-python
import pandas as pd
import mysql.connector

host = 'rhea.isegi.unl.pt'
user = 'wwi-read-only-user'
password = 'jGp2GCqrss6nfTEu5ZawhW3mksLsQYQb'
database = 'WWI'

mydb = mysql.connector.connect(host=host, user=user, database=database, port=3306, password=password)
mycursor = mydb.cursor()

mycursor.execute('SHOW TABLES;')
tables_list = mycursor.fetchall()
print(tables_list)

[('Purchasing_PurchaseOrderLines',), ('Purchasing_PurchaseOrders',), ('Purchasing_SupplierCategories',), ('Purchasing_SupplierTransactions',), ('Purchasing_Suppliers',), ('Sales_CustomerCategories',), ('Sales_CustomerTransactions',), ('Sales_Customers',), ('Sales_InvoiceLines',), ('Sales_Invoices',), ('Sales_OrderLines',), ('Sales_Orders',)]


In [142]:
def fixlist(lista):
    listb = []
    for x in lista:
        for y in x:
            listb.append(y)
    return listb

In [143]:
tables_list2 = fixlist(tables_list)
tables_list2

['Purchasing_PurchaseOrderLines',
 'Purchasing_PurchaseOrders',
 'Purchasing_SupplierCategories',
 'Purchasing_SupplierTransactions',
 'Purchasing_Suppliers',
 'Sales_CustomerCategories',
 'Sales_CustomerTransactions',
 'Sales_Customers',
 'Sales_InvoiceLines',
 'Sales_Invoices',
 'Sales_OrderLines',
 'Sales_Orders']

In [1]:
from pymongo import MongoClient
from bson.objectid import ObjectId
from pprint import pprint

host="rhea.isegi.unl.pt"
port="27039"
user="GROUP_22"
password="AW319u0374I82895091g8k9291991eJV"
protocol="mongodb"
database = 'WWI'
client = MongoClient(f"{protocol}://{user}:{password}@{host}:{port}")

In [2]:
db = client.WWI
print(f"Database info: {db}\n")

Database info: Database(MongoClient(host=['rhea.isegi.unl.pt:27039'], document_class=dict, tz_aware=False, connect=True), 'WWI')



In [ ]:
groupinv = { "$group": { "_id": "$InvoiceID", "TotalInv": { "$sum": '$TotalItemPrice' } } }

project = {'$project': {'_id':1,
                       'TotalInv': 1}}

looktoinv = {'$lookup': { 'from': "Sales_Invoices",
        'localField': "_id",
        'foreignField': "InvoiceID",
        'as': "invoice_info"
    }}

unwind ={"$unwind":"$invoice_info"}

project2 = {'$project': {'_id':0,
                       'invoice_info.InvoiceID': 1,
                        'invoice_info.SalespersonPersonID':1,
                        'TotalInv':1,
                        'invoice_info.InvoiceDate':1}}

convert = { "$addFields": {
    "InvDate": { "$convert": { 
                "input": "$invoice_info.InvoiceDate", 
                "to": "date"}}}}

addf = { "$addFields": {'year' : { '$year' : '$InvDate'}}}

match = {'$match': {'year': {'$eq' : 2013}}}

group_person = {'$group':{
    "_id": "$invoice_info.SalespersonPersonID",
    "TotalInvoices": { "$addToSet":"$TotalInv" }
}}

project3 = {'$project': { '_id': 0,
                         "invoice_info.SalespersonPersonID":1,
                        "MoneyPerson": { '$sum': "$TotalInv" }}}

sortdesc = {'$sort': {'MoneyPerson':-1}}

limit ={"$limit":1}

looktopers = {'$lookup': { 'from': "Project_WWI_Application_People",
        'localField': "invoice_info.SalespersonPersonID",
        'foreignField': "PersonID",
        'as': "person_info"
    }}

unwind2 = {"$unwind":"$person_info"}

finproj = {'$project': { '_id': 0,
                         'PersonName': "$person_info.FullName",
                        "MoneyPerson":1}}


pipeline = [groupinv,project, looktoinv,unwind,project2,convert,addf, match,group_person, project3,
                                      sortdesc, limit, looktopers,unwind2,finproj]

r = db.Sales_InvoiceLines.aggregate(pipeline)
           
list(r)

In [146]:
############################################## SQL Migration ########################################################

# Transform all SQL tables into csv files
#for i in range(len(tables_list2)):
#    query = f'select * from {tables_list2[i]}'
#    results = pd.read_sql_query(query, mydb)
#    table_name = tables_list2[i]
#    results.to_csv(table_name+".csv", index=False)

In [147]:
# Function to transform csv files into dictionaries

def csvtodict(path):
    df = pd.read_csv(path)
    return df.to_dict('records')

In [148]:
# Add the dictionaries as collections to our database
for table in tables_list2:
    db[table].insert_many(csvtodict(table+".csv"))

In [149]:
# check to see if they were properly added
collection_list = db.list_collection_names()
print(f"The database contains {len(collection_list)} collections")

print(f"All collections: {collection_list}")

The database contains 12 collections
All collections: ['Sales_Customers', 'Purchasing_SupplierTransactions', 'Sales_Orders', 'Purchasing_SupplierCategories', 'Sales_CustomerCategories', 'Sales_CustomerTransactions', 'Purchasing_Suppliers', 'Purchasing_PurchaseOrderLines', 'Sales_OrderLines', 'Sales_InvoiceLines', 'Sales_Invoices', 'Purchasing_PurchaseOrders']


In [150]:
# check the fields of each table to match given diagram
db.Sales_Customers.find_one()

{'_id': ObjectId('60b3d7e5781dea0108ea33f1'),
 'CustomerID': 1,
 'CustomerName': 'Tailspin Toys (Head Office)',
 'BillToCustomerID': 1,
 'CustomerCategoryID': 3,
 'PrimaryContactPersonID': 1001,
 'AlternateContactPersonID': 1002.0,
 'DeliveryMethodID': 3,
 'DeliveryCityID': 19586,
 'PostalCityID': 19586,
 'CreditLimit': nan,
 'AccountOpenedDate': '2013-01-01',
 'StandardDiscountPercentage': 0.0,
 'IsStatementSent': 0,
 'IsOnCreditHold': 0,
 'PaymentDays': 7,
 'PhoneNumber': '(308) 555-0100',
 'FaxNumber': '(308) 555-0101',
 'DeliveryRun': nan,
 'RunPosition': nan,
 'WebsiteURL': 'http://www.tailspintoys.com',
 'DeliveryAddressLine1': 'Shop 38',
 'DeliveryAddressLine2': '1877 Mittal Road',
 'DeliveryPostalCode': 90410,
 'DeliveryLocationLat': 41.4972,
 'DeliveryLocationLong': -102.62,
 'PostalAddressLine1': 'PO Box 8975',
 'PostalAddressLine2': 'Ribeiroville',
 'PostalPostalCode': 90410}

In [151]:
########################################## CSV Migration ##########################################################

csvfiles_list = ['Project_WWI_Warehouse_Colors','Project_WWI_Warehouse_PackageTypes','Project_WWI_Warehouse_StockGroups',
                'Project_WWI_Warehouse_StockItems','Project_WWI_Warehouse_StockItemStockGroups','Project_WWI_Warehouse_StockItemTransactions']

for file in csvfiles_list:
    db[file].insert_many(csvtodict(file+".csv"))
    

In [152]:
# check to see if they were properly added
collection_list = db.list_collection_names()
print(f"The database contains {len(collection_list)} collections")

print(f"All collections: {collection_list}")

The database contains 18 collections
All collections: ['Sales_Customers', 'Project_WWI_Warehouse_StockItemStockGroups', 'Project_WWI_Warehouse_StockItemTransactions', 'Purchasing_SupplierTransactions', 'Sales_Orders', 'Purchasing_SupplierCategories', 'Sales_CustomerCategories', 'Sales_CustomerTransactions', 'Project_WWI_Warehouse_PackageTypes', 'Purchasing_Suppliers', 'Purchasing_PurchaseOrderLines', 'Sales_OrderLines', 'Project_WWI_Warehouse_StockItems', 'Sales_InvoiceLines', 'Sales_Invoices', 'Project_WWI_Warehouse_StockGroups', 'Project_WWI_Warehouse_Colors', 'Purchasing_PurchaseOrders']


In [153]:
# check the fields of each table to match given diagram
db.Project_WWI_Warehouse_Colors.find_one()

{'_id': ObjectId('60b3d89d781dea0108ee0781'),
 'ColorID': 1,
 'ColorName': 'Azure'}

In [154]:
############################################## Json Migration ########################################################
import json

jsonfiles_list = ['Project_WWI_Application_Cities.json', 'Project_WWI_Application_Countries.json', 
                  'Project_WWI_Application_DeliveryMethods.json', 'Project_WWI_Application_PaymentMethods.json',
                  'Project_WWI_Application_People.json', 'Project_WWI_Application_StateProvinces.json',
                  'Project_WWI_Application_TransactionTypes.json']

jsonfiles_names = ['Project_WWI_Application_Cities', 'Project_WWI_Application_Countries', 
                  'Project_WWI_Application_DeliveryMethods', 'Project_WWI_Application_PaymentMethods',
                  'Project_WWI_Application_People', 'Project_WWI_Application_StateProvinces',
                  'Project_WWI_Application_TransactionTypes']

for path,name in zip(jsonfiles_list,jsonfiles_names): 
    with open(path) as json_file:
        json_2 = json.load(json_file)
        data = json_2
        db[name].insert_many(data)
       

In [155]:
# check the fields of each table to match given diagram
db.Project_WWI_Application_Countries.find_one()

{'_id': ObjectId('60b3d8f5781dea0108f23b09'),
 'CountryID': 1,
 'CountryName': 'Afghanistan',
 'FormalName': 'Islamic State of Afghanistan',
 'IsoAlpha3Code': 'AFG',
 'IsoNumericCode': 4,
 'CountryType': 'UN Member State',
 'LatestRecordedPopulation': 28400000,
 'Continent': 'Asia',
 'Region': 'Asia',
 'Subregion': 'Southern Asia'}

### Creating The Model:

* First draft of the embedding process from the Relational Diagram:

<img src="./linkedWWI-draft2.jpg" alt="Diagram" style="width: 600px;"/>

In [156]:
#Note: in purchasing suppliers & sales customers it doesnt match the diagram, delivery location is actually 2 fields latitude and longitude

In [157]:
# main function with the standard embedding 
def embedding(principal_collection, lookup_collection, match_field1, match_field2, alias, merge_into, unwind = False):
    query_1 = {
            "$lookup":{
               "from": lookup_collection,
               "localField": match_field1,
               "foreignField": match_field2,
               "as": alias
            }
        }

    query_2 = {
            "$project":{
               alias+"._id": 0
            }
        }

    query_3 = { 
            "$merge" : {
                "into" : merge_into,
                "whenMatched": "merge"
            } 
        }
    
    if unwind:
        unwind_query = {"$unwind": "$"+ alias}
        pipeline = [query_1, unwind_query, query_2, query_3]  
    else:  
        pipeline = [query_1, query_2, query_3]
        
    
    principal_collection.aggregate(pipeline)
    
    
    db[merge_into].update_many(
        { },
        { "$unset": { match_field1: "" } }
    )
    
    return list(db[merge_into].find())
    

In [158]:
# embed colours table into stock items table

r = embedding(principal_collection = db.Project_WWI_Warehouse_StockItems, 
              lookup_collection = "Project_WWI_Warehouse_Colors", match_field1 = "ColorID",
              match_field2 = "ColorID", alias = "Color",
              merge_into = "Warehouse_StockItems_Embeding1", unwind = True)

r[1]

{'_id': ObjectId('60b3d89d781dea0108ee07bf'),
 'Barcode': nan,
 'Brand': nan,
 'Color': {'ColorID': 3, 'ColorName': 'Black'},
 'InternalComments': nan,
 'IsChillerStock': False,
 'LeadTimeDays': 14,
 'MarketingComments': "Need to see over your cubicle wall? This is just what's needed.",
 'OuterPackageID': 6,
 'Photo': nan,
 'QuantityPerOuter': 10,
 'RecommendedRetailPrice': 27.66,
 'SearchDetails': "Office cube periscope (Black) Need to see over your cubicle wall? This is just what's needed.",
 'Size': nan,
 'StockItemID': 3,
 'StockItemName': 'Office cube periscope (Black)',
 'SupplierID': 12,
 'TaxRate': 15.0,
 'TypicalWeightPerUnit': 0.25,
 'UnitPackageID': 7,
 'UnitPrice': 18.5}

In [159]:
# can now drop the unnecessary Project_WWI_Warehouse_Colors table & older Project_WWI_Warehouse_StockItems table
db['Project_WWI_Warehouse_Colors'].drop()
db['Project_WWI_Warehouse_StockItems'].drop()

In [160]:
# embed stock groups into stock item stock groups table

r = embedding(principal_collection = db.Project_WWI_Warehouse_StockItemStockGroups, 
              lookup_collection = "Project_WWI_Warehouse_StockGroups", match_field1 = "StockGroupID",
              match_field2 = "StockGroupID", alias = "Group",
              merge_into = "Warehouse_StockItemsStockGroups_Embeding1", unwind = True)

r[0]

{'_id': ObjectId('60b3d89d781dea0108ee08a0'),
 'Group': {'StockGroupID': 6, 'StockGroupName': 'Computing Novelties'},
 'StockItemID': 1,
 'StockItemStockGroupID': 1}

In [161]:
#can now drop the unnecessary Project_WWI_Warehouse_StockGroups table
db['Project_WWI_Warehouse_StockGroups'].drop()

In [162]:
# take out previous unique key for the relational database and add groupid and name as fields
db.Warehouse_StockItemsStockGroups_Embeding1.aggregate([{
    "$addFields": { "StockGroupID": "$Group.StockGroupID", "StockGroupName": "$Group.StockGroupName"} },
    { 
            "$merge" : {
                "into" : "Warehouse_StockItemsStockGroups_Embeding_Group",
                "whenMatched": "merge"
            } 
        }
    
])
db.Warehouse_StockItemsStockGroups_Embeding_Group.update_many(
        { },
        { "$unset": { 'StockItemStockGroupID': "" , "Group": ""} }
    )

In [163]:
list(db.Warehouse_StockItemsStockGroups_Embeding_Group.find())[8]

{'_id': ObjectId('60b3d89d781dea0108ee08a8'),
 'StockGroupID': 6,
 'StockGroupName': 'Computing Novelties',
 'StockItemID': 4}

In [164]:
db['Warehouse_StockItemsStockGroups_Embeding1'].drop()

In [165]:
# embed stock item stock groups into stock items table
# Given that we need to use a different template this time we cant use the general function we made
query_1 = {
        "$lookup":{
           "from": "Warehouse_StockItemsStockGroups_Embeding_Group",
           "localField": "StockItemID",
           "foreignField": "StockItemID",
           "as": "Groups"
        }
    }

# Here's the difference - instead of unset matching field from the main table
# this time we want to delete from the embeded part
query_2 = {
        "$project":{
           "Groups._id": 0,
           "Groups.StockItemID": 0 
        }
    }

query_3 = { 
        "$merge" : {
            "into" : "Warehouse_StockItems_Embeding2",
            "whenMatched": "merge"
        } 
    }

pipeline = [query_1, query_2, query_3]  

db.Warehouse_StockItems_Embeding1.aggregate(pipeline)


In [166]:
list(db.Warehouse_StockItems_Embeding2.find())[1]

{'_id': ObjectId('60b3d89d781dea0108ee07bf'),
 'Barcode': nan,
 'Brand': nan,
 'Color': {'ColorID': 3, 'ColorName': 'Black'},
 'Groups': [{'StockGroupID': 6, 'StockGroupName': 'Computing Novelties'},
  {'StockGroupID': 1, 'StockGroupName': 'Novelty Items'}],
 'InternalComments': nan,
 'IsChillerStock': False,
 'LeadTimeDays': 14,
 'MarketingComments': "Need to see over your cubicle wall? This is just what's needed.",
 'OuterPackageID': 6,
 'Photo': nan,
 'QuantityPerOuter': 10,
 'RecommendedRetailPrice': 27.66,
 'SearchDetails': "Office cube periscope (Black) Need to see over your cubicle wall? This is just what's needed.",
 'Size': nan,
 'StockItemID': 3,
 'StockItemName': 'Office cube periscope (Black)',
 'SupplierID': 12,
 'TaxRate': 15.0,
 'TypicalWeightPerUnit': 0.25,
 'UnitPackageID': 7,
 'UnitPrice': 18.5}

In [167]:
# embed warehouse package type into stokcitems

r = embedding(principal_collection = db.Warehouse_StockItems_Embeding2, 
              lookup_collection = "Project_WWI_Warehouse_PackageTypes", match_field1 = "UnitPackageID",
              match_field2 = "PackageTypeID", alias = "UnitPackage_Details",
              merge_into = "Warehouse_StockItems_Embeding3", unwind = True)

r = embedding(principal_collection = db.Warehouse_StockItems_Embeding3, 
              lookup_collection = "Project_WWI_Warehouse_PackageTypes", match_field1 = "OuterPackageID",
              match_field2 = "PackageTypeID", alias = "OuterPackage_Details",
              merge_into = "Warehouse_StockItems_Embeding4", unwind = True)
r[1]

{'_id': ObjectId('60b3d89d781dea0108ee07bf'),
 'Barcode': nan,
 'Brand': nan,
 'Color': {'ColorID': 3, 'ColorName': 'Black'},
 'Groups': [{'StockGroupID': 6, 'StockGroupName': 'Computing Novelties'},
  {'StockGroupID': 1, 'StockGroupName': 'Novelty Items'}],
 'InternalComments': nan,
 'IsChillerStock': False,
 'LeadTimeDays': 14,
 'MarketingComments': "Need to see over your cubicle wall? This is just what's needed.",
 'OuterPackage_Details': {'PackageTypeID': 6, 'PackageTypeName': 'Carton'},
 'Photo': nan,
 'QuantityPerOuter': 10,
 'RecommendedRetailPrice': 27.66,
 'SearchDetails': "Office cube periscope (Black) Need to see over your cubicle wall? This is just what's needed.",
 'Size': nan,
 'StockItemID': 3,
 'StockItemName': 'Office cube periscope (Black)',
 'SupplierID': 12,
 'TaxRate': 15.0,
 'TypicalWeightPerUnit': 0.25,
 'UnitPackage_Details': {'PackageTypeID': 7, 'PackageTypeName': 'Each'},
 'UnitPrice': 18.5}

In [168]:
# can now drop the unnecessary Project_WWI_Warehouse_StockItemStockGroups table & 
# old embedded tables 
db['Project_WWI_Warehouse_StockItemStockGroups'].drop()
db['Warehouse_StockItemsStockGroups_Embeding_Group'].drop()
db['Warehouse_StockItems_Embeding1'].drop()
db['Warehouse_StockItems_Embeding2'].drop()
db['Warehouse_StockItems_Embeding3'].drop()

In [169]:
# embed warehouse package type into purchase order lines

r = embedding(principal_collection = db.Purchasing_PurchaseOrderLines, 
              lookup_collection = "Project_WWI_Warehouse_PackageTypes", match_field1 = "PackageTypeID",
              match_field2 = "PackageTypeID", alias = "Package_Details",
              merge_into = "PurchaseOrderLines_Embeding1", unwind=True)

r[1]

{'_id': ObjectId('60b3d7ca781dea0108e93665'),
 'Description': 'Pack of 12 action figures (male)',
 'ExpectedUnitPricePerOuter': 5.5,
 'IsOrderLineFinalized': 1,
 'OrderedOuters': 21,
 'Package_Details': {'PackageTypeID': 9, 'PackageTypeName': 'Packet'},
 'PurchaseOrderID': 1,
 'PurchaseOrderLineID': 2,
 'ReceivedOuters': 21,
 'StockItemID': 151}

In [170]:
#can now drop the unnecessary Purchasing_PurchaseOrderLines table
db['Purchasing_PurchaseOrderLines'].drop()

In [171]:
# embed supplier categories into suppliers
r = embedding(principal_collection = db.Purchasing_Suppliers, 
              lookup_collection = "Purchasing_SupplierCategories", match_field1 = "SupplierCategoryID",
              match_field2 = "SupplierCategoryID", alias = "Supplier_Category",
              merge_into = "Purchasing_Suppliers_Embeding1", unwind= True)

r[1]

{'_id': ObjectId('60b3d7cc781dea0108e968bd'),
 'AlternateContactPersonID': 24,
 'BankAccountBranch': 'Woodgrove Bank Greenbank',
 'BankAccountCode': 358698,
 'BankAccountName': 'Contoso Ltd',
 'BankAccountNumber': 4587965215,
 'BankInternationalCode': 25868,
 'DeliveryAddressLine1': 'Unit 2',
 'DeliveryAddressLine2': '2934 Night Road',
 'DeliveryCityID': 13870,
 'DeliveryLocationLat': 48.0982,
 'DeliveryLocationLong': -122.573,
 'DeliveryMethodID': 9.0,
 'DeliveryPostalCode': 98253,
 'FaxNumber': '(360) 555-0101',
 'InternalComments': nan,
 'PaymentDays': 7,
 'PhoneNumber': '(360) 555-0100',
 'PostalAddressLine1': 'PO Box 1012',
 'PostalAddressLine2': 'Jolimont',
 'PostalCityID': 13870,
 'PostalPostalCode': 98253,
 'PrimaryContactPersonID': 23,
 'SupplierID': 2,
 'SupplierName': 'Contoso, Ltd.',
 'SupplierReference': 'B2084020',
 'Supplier_Category': {'SupplierCategoryID': 2,
  'SupplierCategoryName': 'Novelty Goods Supplier'},
 'WebsiteURL': 'http://www.contoso.com'}

In [172]:
# can now drop the unnecessary Purchasing_SupplierCategories table & older Purchasing_Suppliers table
db['Purchasing_SupplierCategories'].drop()
db['Purchasing_Suppliers'].drop()

In [173]:
# embed warehouse package type into sales order lines
r = embedding(principal_collection = db.Sales_OrderLines, 
              lookup_collection = "Project_WWI_Warehouse_PackageTypes", match_field1 = "PackageTypeID",
              match_field2 = "PackageTypeID", alias = "Package_Details",
              merge_into = "Sales_OrderLines_Embeding1", unwind = True)

r[1]

{'_id': ObjectId('60b3d850781dea0108ebbf27'),
 'Description': 'Ride on toy sedan car (Black) 1/12 scale',
 'OrderID': 1,
 'OrderLineID': 2,
 'Package_Details': {'PackageTypeID': 7, 'PackageTypeName': 'Each'},
 'PickedQuantity': 10,
 'PickingCompletedWhen': '2013-01-01 11:00:00',
 'Quantity': 10,
 'StockItemID': 67,
 'TaxRate': 15.0,
 'UnitPrice': 230.0}

In [174]:
# embed warehouse package type into invoices order lines
r = embedding(principal_collection = db.Sales_InvoiceLines, 
              lookup_collection = "Project_WWI_Warehouse_PackageTypes", match_field1 = "PackageTypeID",
              match_field2 = "PackageTypeID", alias = "Package_Details",
              merge_into = "Sales_InvoiceLines_Embeding1", unwind = True)
r[1]

{'_id': ObjectId('60b3d7e6781dea0108ea3689'),
 'Description': 'Developer joke mug - old C developers never die (White)',
 'ExtendedPrice': 134.55,
 'InvoiceID': 2,
 'InvoiceLineID': 2,
 'LineProfit': 76.5,
 'Package_Details': {'PackageTypeID': 7, 'PackageTypeName': 'Each'},
 'Quantity': 9,
 'StockItemID': 50,
 'TaxAmount': 17.55,
 'TaxRate': 15.0,
 'UnitPrice': 13.0}

In [175]:
# can now drop the unnecessary Project_WWI_Warehouse_PackageTypes table & older Sales_OrderLines table
db['Project_WWI_Warehouse_PackageTypes'].drop()
db['Sales_OrderLines'].drop()
db['Sales_InvoiceLines'].drop()

In [176]:
# embed customer categories into customers
r = embedding(principal_collection = db.Sales_Customers, 
              lookup_collection = "Sales_CustomerCategories", match_field1 = "CustomerCategoryID",
              match_field2 = "CustomerCategoryID", alias = "Customer_Category",
              merge_into = "Sales_Customers_Embeding1", unwind = True)

r[1]

{'_id': ObjectId('60b3d7e5781dea0108ea33f2'),
 'AccountOpenedDate': '2013-01-01',
 'AlternateContactPersonID': 1004.0,
 'BillToCustomerID': 1,
 'CreditLimit': nan,
 'CustomerID': 2,
 'CustomerName': 'Tailspin Toys (Sylvanite, MT)',
 'Customer_Category': {'CustomerCategoryID': 3,
  'CustomerCategoryName': 'Novelty Shop'},
 'DeliveryAddressLine1': 'Shop 245',
 'DeliveryAddressLine2': '705 Dita Lane',
 'DeliveryCityID': 33475,
 'DeliveryLocationLat': 48.7163,
 'DeliveryLocationLong': -115.874,
 'DeliveryMethodID': 3,
 'DeliveryPostalCode': 90216,
 'DeliveryRun': nan,
 'FaxNumber': '(406) 555-0101',
 'IsOnCreditHold': 0,
 'IsStatementSent': 0,
 'PaymentDays': 7,
 'PhoneNumber': '(406) 555-0100',
 'PostalAddressLine1': 'PO Box 259',
 'PostalAddressLine2': 'Jogiville',
 'PostalCityID': 33475,
 'PostalPostalCode': 90216,
 'PrimaryContactPersonID': 1003,
 'RunPosition': nan,
 'StandardDiscountPercentage': 0.0,
 'WebsiteURL': 'http://www.tailspintoys.com/Sylvanite'}

In [177]:
# can now drop the unnecessary Sales_CustomerCategories table & older Sales_Customers table
db['Sales_CustomerCategories'].drop()
db['Sales_Customers'].drop()

In [178]:
# embed transaction types 
transactions = {'Purchasing_SupplierTransactions': db.Purchasing_SupplierTransactions, 
                'Sales_CustomerTransactions': db.Sales_CustomerTransactions, 
                'Warehouse_StockItemTransactions': db.Project_WWI_Warehouse_StockItemTransactions}

for key,item in transactions.items():
    r = embedding(principal_collection = item, 
              lookup_collection = "Project_WWI_Application_TransactionTypes", match_field1 = "TransactionTypeID",
              match_field2 = "TransactionTypeID", alias = "Transaction_Type",
              merge_into = key+"_Embeding1", unwind = True)


In [179]:
list(db.Purchasing_SupplierTransactions_Embeding1.find())[1]

{'_id': ObjectId('60b3d7cc781dea0108e95f37'),
 'AmountExcludingTax': 21732.0,
 'FinalizationDate': '2013-01-07',
 'IsFinalized': 1,
 'OutstandingBalance': 0.0,
 'PaymentMethodID': 4,
 'PurchaseOrderID': 2.0,
 'SupplierID': 4,
 'SupplierInvoiceNumber': 3898.0,
 'SupplierTransactionID': 169,
 'TaxAmount': 3259.8,
 'TransactionAmount': 24991.8,
 'TransactionDate': '2013-01-02',
 'Transaction_Type': {'TransactionTypeID': 5,
  'TransactionTypeName': 'Supplier Invoice'}}

In [180]:
list(db.Sales_CustomerTransactions_Embeding1.find())[1]

{'_id': ObjectId('60b3d7cd781dea0108e968d2'),
 'AmountExcludingTax': 405.0,
 'CustomerID': 803,
 'CustomerTransactionID': 5,
 'FinalizationDate': '2013-01-02',
 'InvoiceID': 2.0,
 'IsFinalized': 1,
 'OutstandingBalance': 0.0,
 'PaymentMethodID': nan,
 'TaxAmount': 60.75,
 'TransactionAmount': 465.75,
 'TransactionDate': '2013-01-01',
 'Transaction_Type': {'TransactionTypeID': 1,
  'TransactionTypeName': 'Customer Invoice'}}

In [181]:
list(db.Warehouse_StockItemTransactions_Embeding1.find())[1]

{'_id': ObjectId('60b3d89f781dea0108ee0a5b'),
 'CustomerID': 803.0,
 'InvoiceID': 2.0,
 'PurchaseOrderID': nan,
 'Quantity': -9.0,
 'StockItemID': 50,
 'StockItemTransactionID': 3,
 'SupplierID': nan,
 'TransactionOccurredWhen': '2013-01-01 12:00:00.0000000',
 'Transaction_Type': {'TransactionTypeID': 10,
  'TransactionTypeName': 'Stock Issue'}}

In [182]:
# can now drop the unnecessary older tables
db['Sales_CustomerTransactions'].drop()
db['Purchasing_SupplierTransactions'].drop()
db['Project_WWI_Application_TransactionTypes'].drop()
db['Project_WWI_Warehouse_StockItemTransactions'].drop()

In [183]:
# embed countries into state provinces

r = embedding(principal_collection = db.Project_WWI_Application_StateProvinces, 
              lookup_collection = "Project_WWI_Application_Countries", match_field1 = "CountryID",
              match_field2 = "CountryID", alias = "Country_Details",
              merge_into = "Application_StateProvinces_Embeding1", unwind = True)

r[1]

{'_id': ObjectId('60b3d8f6781dea0108f2402d'),
 'Country_Details': {'CountryID': 230,
  'CountryName': 'United States',
  'FormalName': 'United States of America',
  'IsoAlpha3Code': 'USA',
  'IsoNumericCode': 840,
  'CountryType': 'UN Member State',
  'LatestRecordedPopulation': 313973000,
  'Continent': 'North America',
  'Region': 'Americas',
  'Subregion': 'Northern America'},
 'LatestRecordedPopulation': 735132,
 'SalesTerritory': 'Far West',
 'StateProvinceCode': 'AK',
 'StateProvinceID': 2,
 'StateProvinceName': 'Alaska'}

In [184]:
# can now drop the unnecessary Project_WWI_Application_Countries table & older Project_WWI_Application_StateProvinces table
db['Project_WWI_Application_Countries'].drop()
db['Project_WWI_Application_StateProvinces'].drop()

In [185]:
# embed state provinces into cities
r = embedding(principal_collection = db.Project_WWI_Application_Cities, 
              lookup_collection = "Application_StateProvinces_Embeding1", match_field1 = "StateProvinceID",
              match_field2 = "StateProvinceID", alias = "State_Province_Details",
              merge_into = "Application_Cities_Embeding1", unwind = True)

r[1]

{'_id': ObjectId('60b3d8ed781dea0108f1a6d6'),
 'CityID': 3,
 'CityName': 'Abanda',
 'LatestRecordedPopulation': 192,
 'Location': '0xE6100000010C4033880FEC8C4040EFBF3A33E66155C0',
 'State_Province_Details': {'Country_Details': {'CountryID': 230,
   'CountryName': 'United States',
   'FormalName': 'United States of America',
   'IsoAlpha3Code': 'USA',
   'IsoNumericCode': 840,
   'CountryType': 'UN Member State',
   'LatestRecordedPopulation': 313973000,
   'Continent': 'North America',
   'Region': 'Americas',
   'Subregion': 'Northern America'},
  'LatestRecordedPopulation': 5437278,
  'SalesTerritory': 'Southeast',
  'StateProvinceCode': 'AL',
  'StateProvinceID': 1,
  'StateProvinceName': 'Alabama'}}

In [186]:
#can now drop the unnecessary Application_StateProvinces_Embeding1 table & older Project_WWI_Application_Cities table
db['Application_StateProvinces_Embeding1'].drop()
db['Project_WWI_Application_Cities'].drop()

In [187]:
# embed cities into customers

r = embedding(principal_collection = db.Sales_Customers_Embeding1, 
              lookup_collection = "Application_Cities_Embeding1", match_field1 = "DeliveryCityID",
              match_field2 = "CityID", alias = "DeliveryCity_Details",
              merge_into = "Sales_Customers_Embeding2", unwind = True)

r = embedding(principal_collection = db.Sales_Customers_Embeding2, 
              lookup_collection = "Application_Cities_Embeding1", match_field1 = "PostalCityID",
              match_field2 = "CityID", alias = "PostalCity_Details",
              merge_into = "Sales_Customers_Embeding3", unwind = True)
r[1]

{'_id': ObjectId('60b3d7e5781dea0108ea33f2'),
 'AccountOpenedDate': '2013-01-01',
 'AlternateContactPersonID': 1004.0,
 'BillToCustomerID': 1,
 'CreditLimit': nan,
 'CustomerID': 2,
 'CustomerName': 'Tailspin Toys (Sylvanite, MT)',
 'Customer_Category': {'CustomerCategoryID': 3,
  'CustomerCategoryName': 'Novelty Shop'},
 'DeliveryAddressLine1': 'Shop 245',
 'DeliveryAddressLine2': '705 Dita Lane',
 'DeliveryCity_Details': {'CityID': 33475,
  'CityName': 'Sylvanite',
  'LatestRecordedPopulation': 103,
  'Location': '0xE6100000010CF37A8BE2B05B4840056FA35CF5F75CC0',
  'State_Province_Details': {'Country_Details': {'CountryID': 230,
    'CountryName': 'United States',
    'FormalName': 'United States of America',
    'IsoAlpha3Code': 'USA',
    'IsoNumericCode': 840,
    'CountryType': 'UN Member State',
    'LatestRecordedPopulation': 313973000,
    'Continent': 'North America',
    'Region': 'Americas',
    'Subregion': 'Northern America'},
   'LatestRecordedPopulation': 1187597,
   'Sa

In [188]:
# can now drop the older Sales_Customers_Embeding1 table
db['Sales_Customers_Embeding1'].drop()
db['Sales_Customers_Embeding2'].drop()

In [189]:
# embed cities into suppliers

r = embedding(principal_collection = db.Purchasing_Suppliers_Embeding1, 
              lookup_collection = "Application_Cities_Embeding1", match_field1 = "DeliveryCityID",
              match_field2 = "CityID", alias = "DeliveryCity_Details",
              merge_into = "Purchasing_Suppliers_Embeding2", unwind = True)

r = embedding(principal_collection = db.Purchasing_Suppliers_Embeding2, 
              lookup_collection = "Application_Cities_Embeding1", match_field1 = "PostalCityID",
              match_field2 = "CityID", alias = "PostalCity_Details",
              merge_into = "Purchasing_Suppliers_Embeding3", unwind = True)
r[1]

{'_id': ObjectId('60b3d7cc781dea0108e968bd'),
 'AlternateContactPersonID': 24,
 'BankAccountBranch': 'Woodgrove Bank Greenbank',
 'BankAccountCode': 358698,
 'BankAccountName': 'Contoso Ltd',
 'BankAccountNumber': 4587965215,
 'BankInternationalCode': 25868,
 'DeliveryAddressLine1': 'Unit 2',
 'DeliveryAddressLine2': '2934 Night Road',
 'DeliveryCity_Details': {'CityID': 13870,
  'CityName': 'Greenbank',
  'LatestRecordedPopulation': None,
  'Location': '0xE6100000010CDA4B6430900C4840C04EFBF7AAA45EC0',
  'State_Province_Details': {'Country_Details': {'CountryID': 230,
    'CountryName': 'United States',
    'FormalName': 'United States of America',
    'IsoAlpha3Code': 'USA',
    'IsoNumericCode': 840,
    'CountryType': 'UN Member State',
    'LatestRecordedPopulation': 313973000,
    'Continent': 'North America',
    'Region': 'Americas',
    'Subregion': 'Northern America'},
   'LatestRecordedPopulation': 7540272,
   'SalesTerritory': 'Far West',
   'StateProvinceCode': 'WA',
   'St

In [190]:
# can now drop the unnecessary embeding tables
db['Application_Cities_Embeding1'].drop()
db['Purchasing_Suppliers_Embeding1'].drop()
db['Application_Cities_Embeding2'].drop()
db['Purchasing_Suppliers_Embeding2'].drop()

In [191]:
# embed payment methods into supplier transactions
r = embedding(principal_collection = db.Purchasing_SupplierTransactions_Embeding1, 
              lookup_collection = "Project_WWI_Application_PaymentMethods", match_field1 = "PaymentMethodID",
              match_field2 = "PaymentMethodID", alias = "Payment_Method",
              merge_into = "Purchasing_SupplierTransactions_Embeding2")

r[1]

{'_id': ObjectId('60b3d7cc781dea0108e95f37'),
 'AmountExcludingTax': 21732.0,
 'FinalizationDate': '2013-01-07',
 'IsFinalized': 1,
 'OutstandingBalance': 0.0,
 'Payment_Method': [{'PaymentMethodID': 4, 'PaymentMethodName': 'EFT'}],
 'PurchaseOrderID': 2.0,
 'SupplierID': 4,
 'SupplierInvoiceNumber': 3898.0,
 'SupplierTransactionID': 169,
 'TaxAmount': 3259.8,
 'TransactionAmount': 24991.8,
 'TransactionDate': '2013-01-02',
 'Transaction_Type': {'TransactionTypeID': 5,
  'TransactionTypeName': 'Supplier Invoice'}}

In [192]:
# can now drop the older Purchasing_SupplierTransactions_Embeding1 table
db['Purchasing_SupplierTransactions_Embeding1'].drop()

In [193]:
# embed payment methods into customer transactions
r = embedding(principal_collection = db.Sales_CustomerTransactions_Embeding1, 
              lookup_collection = "Project_WWI_Application_PaymentMethods", match_field1 = "PaymentMethodID",
              match_field2 = "PaymentMethodID", alias = "Payment_Method",
              merge_into = "Sales_CustomerTransactions_Embeding2")

r[0]

{'_id': ObjectId('60b3d7cd781dea0108e968d1'),
 'AmountExcludingTax': 2300.0,
 'CustomerID': 832,
 'CustomerTransactionID': 2,
 'FinalizationDate': '2013-01-02',
 'InvoiceID': 1.0,
 'IsFinalized': 1,
 'OutstandingBalance': 0.0,
 'Payment_Method': [],
 'TaxAmount': 345.0,
 'TransactionAmount': 2645.0,
 'TransactionDate': '2013-01-01',
 'Transaction_Type': {'TransactionTypeID': 1,
  'TransactionTypeName': 'Customer Invoice'}}

In [194]:
###### can now drop the unnecessary Project_WWI_Application_PaymentMethods table & older PPurchasing_Suppliers_Embeding1 table
db['Project_WWI_Application_PaymentMethods'].drop()
db['Sales_CustomerTransactions_Embeding1'].drop()

In [195]:
# embed delivery methods into purchase orders
r = embedding(principal_collection = db.Purchasing_PurchaseOrders, 
              lookup_collection = "Project_WWI_Application_DeliveryMethods", match_field1 = "DeliveryMethodID",
              match_field2 = "DeliveryMethodID", alias = "Delivery_Method",
              merge_into = "Purchasing_PurchaseOrders_Embeding1")

r[1]

{'_id': ObjectId('60b3d7cb781dea0108e95714'),
 'Comments': nan,
 'ContactPersonID': 2,
 'Delivery_Method': [{'DeliveryMethodID': 7,
   'DeliveryMethodName': 'Road Freight'}],
 'ExpectedDeliveryDate': '2013-01-15',
 'InternalComments': nan,
 'IsOrderFinalized': 1,
 'OrderDate': '2013-01-01',
 'PurchaseOrderID': 2,
 'SupplierID': 4,
 'SupplierReference': '293092'}

In [196]:
# can now drop the older Purchasing_PurchaseOrders_Embeding1 table
db['Purchasing_PurchaseOrders'].drop()

In [197]:
# embed delivery methods into purchase suppliers
r = embedding(principal_collection = db.Purchasing_Suppliers_Embeding3, 
              lookup_collection = "Project_WWI_Application_DeliveryMethods", match_field1 = "DeliveryMethodID",
              match_field2 = "DeliveryMethodID", alias = "Delivery_Method",
              merge_into = "Purchasing_Suppliers_Embeding4")

r[1]

{'_id': ObjectId('60b3d7cc781dea0108e968bd'),
 'AlternateContactPersonID': 24,
 'BankAccountBranch': 'Woodgrove Bank Greenbank',
 'BankAccountCode': 358698,
 'BankAccountName': 'Contoso Ltd',
 'BankAccountNumber': 4587965215,
 'BankInternationalCode': 25868,
 'DeliveryAddressLine1': 'Unit 2',
 'DeliveryAddressLine2': '2934 Night Road',
 'DeliveryCity_Details': {'CityID': 13870,
  'CityName': 'Greenbank',
  'LatestRecordedPopulation': None,
  'Location': '0xE6100000010CDA4B6430900C4840C04EFBF7AAA45EC0',
  'State_Province_Details': {'Country_Details': {'CountryID': 230,
    'CountryName': 'United States',
    'FormalName': 'United States of America',
    'IsoAlpha3Code': 'USA',
    'IsoNumericCode': 840,
    'CountryType': 'UN Member State',
    'LatestRecordedPopulation': 313973000,
    'Continent': 'North America',
    'Region': 'Americas',
    'Subregion': 'Northern America'},
   'LatestRecordedPopulation': 7540272,
   'SalesTerritory': 'Far West',
   'StateProvinceCode': 'WA',
   'St

In [198]:
# can now drop the older Purchasing_Suppliers_Embeding3 table
db['Purchasing_Suppliers_Embeding3'].drop()

In [199]:
# embed delivery methods into sales invoices
r = embedding(principal_collection = db.Sales_Invoices, 
              lookup_collection = "Project_WWI_Application_DeliveryMethods", match_field1 = "DeliveryMethodID",
              match_field2 = "DeliveryMethodID", alias = "Delivery_Method",
              merge_into = "Sales_Invoices_Embeding1")

r[1]

{'_id': ObjectId('60b3d7f2781dea0108eaabb9'),
 'AccountsPersonID': 3003,
 'BillToCustomerID': 803,
 'Comments': nan,
 'ConfirmedDeliveryTime': '2013-01-02 07:10:00',
 'ConfirmedReceivedBy': 'Bala Dixit',
 'ContactPersonID': 3003,
 'CreditNoteReason': nan,
 'CustomerID': 803,
 'CustomerPurchaseOrderNumber': 15342,
 'DeliveryInstructions': 'Unit 7, 844 Magnusson Lane',
 'DeliveryRun': nan,
 'Delivery_Method': [{'DeliveryMethodID': 3,
   'DeliveryMethodName': 'Delivery Van'}],
 'InternalComments': nan,
 'InvoiceDate': '2013-01-01',
 'InvoiceID': 2,
 'IsCreditNote': 0,
 'OrderID': 2,
 'PackedByPersonID': 14,
 'ReturnedDeliveryData': '{"Events": [{ "Event":"Ready for collection","EventTime":"2013-01-01T12:00:00","ConNote":"EAN-125-1052"},{ "Event":"DeliveryAttempt","EventTime":"2013-01-02T07:10:00","ConNote":"EAN-125-1052","DriverID":15,"Latitude":40.8657308,"Longitude":-120.4602262,"Status":"Delivered"}],"DeliveredWhen":"2013-01-02T07:10:00","ReceivedBy":"Bala Dixit"}',
 'RunPosition': nan

In [200]:
# can now drop the older Sales_Invoices_Embeding1 table
db['Sales_Invoices'].drop()

In [201]:
# embed delivery methods into sales customers
r = embedding(principal_collection = db.Sales_Customers_Embeding3, 
              lookup_collection = "Project_WWI_Application_DeliveryMethods", match_field1 = "DeliveryMethodID",
              match_field2 = "DeliveryMethodID", alias = "Delivery_Method",
              merge_into = "Sales_Customers_Embeding4")

r[1]

{'_id': ObjectId('60b3d7e5781dea0108ea33f2'),
 'AccountOpenedDate': '2013-01-01',
 'AlternateContactPersonID': 1004.0,
 'BillToCustomerID': 1,
 'CreditLimit': nan,
 'CustomerID': 2,
 'CustomerName': 'Tailspin Toys (Sylvanite, MT)',
 'Customer_Category': {'CustomerCategoryID': 3,
  'CustomerCategoryName': 'Novelty Shop'},
 'DeliveryAddressLine1': 'Shop 245',
 'DeliveryAddressLine2': '705 Dita Lane',
 'DeliveryCity_Details': {'CityID': 33475,
  'CityName': 'Sylvanite',
  'LatestRecordedPopulation': 103,
  'Location': '0xE6100000010CF37A8BE2B05B4840056FA35CF5F75CC0',
  'State_Province_Details': {'Country_Details': {'CountryID': 230,
    'CountryName': 'United States',
    'FormalName': 'United States of America',
    'IsoAlpha3Code': 'USA',
    'IsoNumericCode': 840,
    'CountryType': 'UN Member State',
    'LatestRecordedPopulation': 313973000,
    'Continent': 'North America',
    'Region': 'Americas',
    'Subregion': 'Northern America'},
   'LatestRecordedPopulation': 1187597,
   'Sa

In [202]:
# can now drop the older Sales_Customers_Embeding3 table
db['Sales_Customers_Embeding3'].drop()

In [203]:
# can now drop the unnecessary Project_WWI_Application_DeliveryMethods
db['Project_WWI_Application_DeliveryMethods'].drop()

In [204]:
# check to see if they all embeddings were done correctly
collection_list = db.list_collection_names()
print(f"The database contains {len(collection_list)} collections")

print(f"All collections: {collection_list}")

The database contains 13 collections
All collections: ['Project_WWI_Application_People', 'Sales_Orders', 'PurchaseOrderLines_Embeding1', 'Sales_OrderLines_Embeding1', 'Sales_InvoiceLines_Embeding1', 'Purchasing_PurchaseOrders_Embeding1', 'Sales_Invoices_Embeding1', 'Purchasing_SupplierTransactions_Embeding2', 'Warehouse_StockItemTransactions_Embeding1', 'Purchasing_Suppliers_Embeding4', 'Warehouse_StockItems_Embeding4', 'Sales_CustomerTransactions_Embeding2', 'Sales_Customers_Embeding4']


In [205]:
#for i in collection_list: db[i].drop()

In [206]:
# Rename tables to their "original" name
db.Purchasing_Suppliers_Embeding4.rename("Purchasing_Suppliers")
db.Sales_Customers_Embeding4.rename("Sales_Customers")
db.PurchaseOrderLines_Embeding1.rename("Purchasing_PurchaseOrderLines")
db.Warehouse_StockItemTransactions_Embeding1.rename("Warehouse_StockItemTransactions")
db.Sales_OrderLines_Embeding1.rename("Sales_OrderLines")
db.Sales_CustomerTransactions_Embeding2.rename("Sales_CustomerTransactions")
db.Purchasing_PurchaseOrders_Embeding1.rename("Purchasing_PurchaseOrders")
db.Warehouse_StockItems_Embeding4.rename("Warehouse_StockItems")
db.Sales_Invoices_Embeding1.rename("Sales_Invoices")
db.Purchasing_SupplierTransactions_Embeding2.rename("Purchasing_SupplierTransactions")
db.Sales_InvoiceLines_Embeding1.rename("Sales_InvoiceLines")

{'ok': 1.0}

### Extra Personalization

In [207]:
# Merge informations about the same thing into only one field - Sales Customers 
query_1 ={
    "$project": {
            "Delivery": {
                "AddressLine1": "$DeliveryAddressLine1", 
                "AddressLine2": "$DeliveryAddressLine2",
                "PostalCode": "$DeliveryPostalCode",
                "Run": "$DeliveryRun"
            },
            "Postal": {
                "AddressLine1": "$PostalAddressLine1", 
                "AddressLine2": "$PostalAddressLine2",
                "PostalCode": "$PostalPostalCode"
            }
        }
}

query_2 = { 
        "$merge" : {
            "into" : "Sales_Customers1",
            "whenMatched": "merge"
        } 
    }

db.Sales_Customers.aggregate([query_1,query_2])

In [225]:
query_1 = {
        "$lookup":{
           "from": "Sales_Customers1",
           "localField": "_id",
           "foreignField": "_id",
           "as": "extra_info"
        }
    }

unwind = {'$unwind': '$extra_info'}

query_2 = {
        "$project":{
           "extra_info._id": 0
        }
    }

query_3 ={
    "$addFields": { "Delivery": "$extra_info.Delivery", "Postal": "$extra_info.Postal"}
        }

query_4 = { 
        "$merge" : {
            "into" : "Sales_Customers2",
            "whenMatched": "merge"
        } 
    }

pipeline = [query_1, query_2, unwind, query_3, query_4]

r = db.Sales_Customers.aggregate(pipeline)

In [226]:
list(db.Sales_Customers2.find())[0]

{'_id': ObjectId('60b3d7e5781dea0108ea33f1'),
 'AccountOpenedDate': '2013-01-01',
 'AlternateContactPersonID': 1002.0,
 'BillToCustomerID': 1,
 'CreditLimit': nan,
 'CustomerID': 1,
 'CustomerName': 'Tailspin Toys (Head Office)',
 'Customer_Category': {'CustomerCategoryID': 3,
  'CustomerCategoryName': 'Novelty Shop'},
 'Delivery': {'AddressLine1': 'Shop 38',
  'AddressLine2': '1877 Mittal Road',
  'PostalCode': 90410,
  'Run': nan},
 'DeliveryAddressLine1': 'Shop 38',
 'DeliveryAddressLine2': '1877 Mittal Road',
 'DeliveryCity_Details': {'CityID': 19586,
  'CityName': 'Lisco',
  'LatestRecordedPopulation': None,
  'Location': '0xE6100000010CE73F5A52A4BF444010638852B1A759C0',
  'State_Province_Details': {'Country_Details': {'CountryID': 230,
    'CountryName': 'United States',
    'FormalName': 'United States of America',
    'IsoAlpha3Code': 'USA',
    'IsoNumericCode': 840,
    'CountryType': 'UN Member State',
    'LatestRecordedPopulation': 313973000,
    'Continent': 'North Americ

In [227]:
# delete previous fields
db.Sales_Customers2.update_many(
   {},
    {"$unset": { "DeliveryAddressLine1": "", "DeliveryAddressLine2": "", 
                          "DeliveryPostalCode": "", 
                          "DeliveryRun": "", 
                          "PostalAddressLine1": "", "PostalAddressLine2": "", 
                          "PostalPostalCode": "", "extra_info": ""} }
)

In [228]:
r = list(db.Sales_Customers2.find({}))

r[0]

{'_id': ObjectId('60b3d7e5781dea0108ea33f1'),
 'AccountOpenedDate': '2013-01-01',
 'AlternateContactPersonID': 1002.0,
 'BillToCustomerID': 1,
 'CreditLimit': nan,
 'CustomerID': 1,
 'CustomerName': 'Tailspin Toys (Head Office)',
 'Customer_Category': {'CustomerCategoryID': 3,
  'CustomerCategoryName': 'Novelty Shop'},
 'Delivery': {'AddressLine1': 'Shop 38',
  'AddressLine2': '1877 Mittal Road',
  'PostalCode': 90410,
  'Run': nan},
 'DeliveryCity_Details': {'CityID': 19586,
  'CityName': 'Lisco',
  'LatestRecordedPopulation': None,
  'Location': '0xE6100000010CE73F5A52A4BF444010638852B1A759C0',
  'State_Province_Details': {'Country_Details': {'CountryID': 230,
    'CountryName': 'United States',
    'FormalName': 'United States of America',
    'IsoAlpha3Code': 'USA',
    'IsoNumericCode': 840,
    'CountryType': 'UN Member State',
    'LatestRecordedPopulation': 313973000,
    'Continent': 'North America',
    'Region': 'Americas',
    'Subregion': 'Northern America'},
   'LatestRec

In [229]:
# drop auxiliar collections
db['Sales_Customers'].drop()
db['Sales_Customers1'].drop()

In [230]:
#### rename 2 back to normal
db.Sales_Customers2.rename("Sales_Customers")

{'ok': 1.0}

In [231]:
# Merge informations about the same thing into only one field - Purchasing Suppliers
query_1 ={
        "$project": {
            "BankAccount": {
                "Name": "$BankAccountName", 
                "Branch": "$BankAccountBranch",
                "Code": "$BankAccountCode",
                "Number": "$BankAccountNumber"
            },
            "Delivery": {
                "AddressLine1": "$DeliveryAddressLine1", 
                "AddressLine2": "$DeliveryAddressLine2",
                "PostalCode": "$DeliveryPostalCode"
            },
            "Postal": {
                "AddressLine1": "$PostalAddressLine1", 
                "AddressLine2": "$PostalAddressLine2",
                "PostalCode": "$PostalPostalCode"
            }
        }
    }

query_2 = { 
        "$merge" : {
            "into" : "Purchasing_Suppliers1",
            "whenMatched": "merge"
        } 
    }

db.Purchasing_Suppliers.aggregate([query_1,query_2])

In [232]:
query_1 = {
        "$lookup":{
           "from": "Purchasing_Suppliers1",
           "localField": "_id",
           "foreignField": "_id",
           "as": "extra_info"
        }
    }

unwind = {'$unwind': '$extra_info'}

query_2 = {
        "$project":{
           "extra_info._id": 0
        }
    }

query_3 ={
    "$addFields": { "BankAccount": "$extra_info.BankAccount", "Delivery": "$extra_info.Delivery", "Postal": "$extra_info.Postal"} }

query_4 = { 
        "$merge" : {
            "into" : "Purchasing_Suppliers2",
            "whenMatched": "merge"
        } 
    }

pipeline = [query_1, query_2, unwind, query_3, query_4]

r = db.Purchasing_Suppliers.aggregate(pipeline)

In [233]:
# delete previous fields
db.Purchasing_Suppliers2.update_many(
   {},
   {"$unset": {"BankAccountName":"", "BankAccountBranch":"",
               "BankAccountCode":"", "BankAccountNumber":"",
               "DeliveryAddressLine1": "", "DeliveryAddressLine2": "", 
               "DeliveryPostalCode": "", "DeliveryLocationLat": "", "DeliveryLocationLong": "", 
               "PostalAddressLine1": "", "PostalAddressLine2": "", 
               "PostalPostalCode": "", "extra_info": ""} }
)

In [234]:
r = list(db.Purchasing_Suppliers2.find({}))

r[0]

{'_id': ObjectId('60b3d7cc781dea0108e968bc'),
 'AlternateContactPersonID': 22,
 'BankAccount': {'Name': 'A Datum Corporation',
  'Branch': 'Woodgrove Bank Zionsville',
  'Code': 356981,
  'Number': 8575824136},
 'BankInternationalCode': 25986,
 'Delivery': {'AddressLine1': 'Suite 10',
  'AddressLine2': '183838 Southwest Boulevard',
  'PostalCode': 46077},
 'DeliveryCity_Details': {'CityID': 38171,
  'CityName': 'Zionsville',
  'LatestRecordedPopulation': 14160,
  'Location': '0xE6100000010CDE115F37B6F9434031276893C39055C0',
  'State_Province_Details': {'Country_Details': {'CountryID': 230,
    'CountryName': 'United States',
    'FormalName': 'United States of America',
    'IsoAlpha3Code': 'USA',
    'IsoNumericCode': 840,
    'CountryType': 'UN Member State',
    'LatestRecordedPopulation': 313973000,
    'Continent': 'North America',
    'Region': 'Americas',
    'Subregion': 'Northern America'},
   'LatestRecordedPopulation': 6570902,
   'SalesTerritory': 'Great Lakes',
   'StatePr

In [235]:
# drop auxiliar collections
db['Purchasing_Suppliers'].drop()
db['Purchasing_Suppliers1'].drop()

In [236]:
#### rename 2 back to normal
db.Purchasing_Suppliers2.rename("Purchasing_Suppliers")

{'ok': 1.0}

### Add new fields with useful information to retrieve 

In [237]:
# Total price by item multiplying the fields unitprice * quantity
db.Sales_OrderLines.aggregate( [
   {
     "$addFields": { "TotalItemPrice":
       { "$multiply": [ "$UnitPrice", "$Quantity" ] } }
   },
    
    { 
        "$merge" : {
            "into" : "Sales_OrdersLines2",
            "whenMatched": "merge"
        } 
    }

] )

db.Sales_InvoiceLines.aggregate( [
   {
     "$addFields": { "TotalItemPrice":
       { "$multiply": [ "$UnitPrice", "$Quantity" ] } }
   },
    { 
        "$merge" : {
            "into" : "Sales_InvoicesLines2",
            "whenMatched": "merge"
        } 
    }
] )

In [238]:
# drop previous ones
db['Sales_InvoiceLines'].drop()
db['Sales_OrderLines'].drop()

In [239]:
#### rename news to normal
db.Sales_OrdersLines2.rename("Sales_OrderLines")
db.Sales_InvoicesLines2.rename("Sales_InvoiceLines")

{'ok': 1.0}

### Create one new collection with all the transactions from customers and suppliers

In [240]:
db.Sales_CustomerTransactions.aggregate([
    {"$project":{
        "_id": 1,
       "Payment_Method": 1,
       "TransactionAmount": 1,
       "TransactionDate": 1,
       "Transaction_Type": 1}
            
        },
    { 
        "$merge" : {
            "into" : "Transactions",
            "whenMatched": "merge"
        } 
    }
])

In [241]:
db.Purchasing_SupplierTransactions.aggregate([
    {"$project":{
        "_id": 1,
       "Payment_Method": 1,
       "TransactionAmount": 1,
       "TransactionDate": 1,
       "Transaction_Type": 1}
            
        },
    { 
        "$merge" : {
            "into" : "Transactions",
            "whenMatched": "merge"
        } 
    }
])

In [242]:
list(db.Transactions.find())[1]

{'_id': ObjectId('60b3d7cd781dea0108e968d2'),
 'Payment_Method': [],
 'TransactionAmount': 465.75,
 'TransactionDate': '2013-01-01',
 'Transaction_Type': {'TransactionTypeID': 1,
  'TransactionTypeName': 'Customer Invoice'}}

In [243]:
list(db.Transactions.find())[52000]

{'_id': ObjectId('60b3d7cc781dea0108e95f36'),
 'Payment_Method': [{'PaymentMethodID': 4, 'PaymentMethodName': 'EFT'}],
 'TransactionAmount': 360.53,
 'TransactionDate': '2013-01-02',
 'Transaction_Type': {'TransactionTypeID': 5,
  'TransactionTypeName': 'Supplier Invoice'}}

In [244]:
# check to see if they all embeddings were done correctly
collection_list = db.list_collection_names()
print(f"The database contains {len(collection_list)} collections")

print(f"All collections: {collection_list}")

The database contains 14 collections
All collections: ['Project_WWI_Application_People', 'Purchasing_Suppliers', 'Sales_Orders', 'Purchasing_PurchaseOrderLines', 'Sales_Customers', 'Purchasing_PurchaseOrders', 'Sales_Invoices', 'Purchasing_SupplierTransactions', 'Warehouse_StockItemTransactions', 'Transactions', 'Sales_InvoiceLines', 'Warehouse_StockItems', 'Sales_CustomerTransactions', 'Sales_OrderLines']
